<a href="https://colab.research.google.com/github/Kalana-Lakshan/Kaggle_Competitions_House-Prices---Advanced-Regression-Techniques/blob/main/House_Prices_Advanced_Regression_Techniques_Advanced_techniques_used.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
!pip install scikit-learn
import sklearn

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
y = np.log1p(train_data.SalePrice)

In [4]:
X = train_data.drop('SalePrice',axis = 1)

In [5]:
categorical_cols = [cname for cname in X.columns if X[cname].nunique()<10 and X[cname].dtype == 'object']
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['float64','int64']]

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

In [8]:
numerical_transformer = SimpleImputer(strategy = 'median')
categorical_transformer = Pipeline(steps = [('imputer',SimpleImputer(strategy = 'most_frequent')),('onehot',OneHotEncoder(handle_unknown = 'ignore'))])
preprocessor = ColumnTransformer(transformers =[('cat',categorical_transformer,categorical_cols),('num',numerical_transformer,numerical_cols)])


In [11]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

my_pipeline5 = Pipeline([
    ('preprocessor', preprocessor),
    ("model", RandomForestRegressor(random_state=42))
])

param_dist = {
    "model__n_estimators": [200, 400, 600, 800],
    "model__max_depth": [None, 10, 20, 30, 50],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ["sqrt", 0.3, 0.5],
    "model__bootstrap": [True]
}


random_search = RandomizedSearchCV(
    my_pipeline5,
    param_dist,
    n_iter=20,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

X_train, X_test, y_train, y_test = train_test_split(X, y)
random_search.fit(X_train, y_train)

best_rf = random_search.best_estimator_
best_rf.fit(X,y)
predictions5 = np.expm1(best_rf.predict(test_data))


In [12]:
output5 = pd.DataFrame({'Id':test_data.Id,'SalePrice':predictions5})
output5.to_csv('submission5.csv',index = False)